In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, col

StatementMeta(, 388358db-7a0a-4f49-940f-8b2263405b23, 3, Finished, Available, Finished)

In [2]:
# 1️⃣ Read from Silver
carts = spark.read.format("delta").table("SilverLakehouse.dbo.slv_carts")
customers = spark.read.format("delta").table("SilverLakehouse.dbo.slv_customers")
products = spark.read.format("delta").table("SilverLakehouse.dbo.slv_products")

# 2️⃣ Join with dimensions
fact_carts = (
    carts.alias("ct")
    .join(customers.alias("c"), col("ct.Customer_ID") == col("c.Customer_ID"), "left")
    .join(products.alias("p"), col("ct.Product_ID") == col("p.Product_ID"), "left")
    .select(
        col("ct.Cart_ID"),
        col("ct.Customer_ID"),
        col("ct.Product_ID"),
        col("ct.Quantity"),
        col("ct.Price"),
        col("ct.Cart_Line_Total"),
        col("ct.DiscountPercentage"),
        col("ct.Cart_Total"),
        col("ct.Cart_Discounted_Total"),
        col("p.Category").alias("Product_Category"),
        col("p.Brand").alias("Product_Brand"),
        col("c.Gender").alias("Customer_Gender"),
        col("c.Age").alias("Customer_Age"),
        col("ct.ingestion_time")
    )
)




StatementMeta(, 388358db-7a0a-4f49-940f-8b2263405b23, 4, Finished, Available, Finished)

In [3]:
# 3️⃣ Add gold ingestion timestamp
fact_carts = fact_carts.withColumn("gold_ingestion_time", current_timestamp())

# 4️⃣ Save as Fact Table
fact_carts.write.format("delta").mode("overwrite").saveAsTable("Fact_Carts")


StatementMeta(, 388358db-7a0a-4f49-940f-8b2263405b23, 5, Finished, Available, Finished)